<p>The intention is that the longest subroute for each line will be used for modelling, as this will cover the greatest number of stops on that line. However, we must check if the longest subroute is also the most common subroute beacause in the case that it is not, there might be a lack of data for the longest subroute.</p>

In [6]:
import pandas as pd
import sqlite3
import numpy as np

conn = sqlite3.connect("busdata_18.sqlite")
cur = conn.cursor()

In [2]:
# Query database
q = "SELECT * FROM trips_cleaned;"
trips_df = pd.read_sql_query(q, conn)

In [3]:
# trips_df contains cleaned data from RT_TRIPS
trips_df.head()

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP
0,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0
1,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN
2,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0
3,07-FEB-18 00:00:00,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0
4,07-FEB-18 00:00:00,6253175,14,14_15,1,85383,81600,84682.0,81608.0


In [4]:
# Create list of lines
all_lines = trips_df.LINEID.unique().tolist()
all_lines.sort()

In [5]:
pd.read_csv("~/data/stoptimes/1_stops.csv").head()

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDDEP_STOP,PLANNEDARR_STOP,ACTUALARR_STOP,ACTUALDEP_STOP
0,02-JAN-18 00:00:00,5956084,43,226,39067,39067,38844,38844
1,02-JAN-18 00:00:00,5956084,42,225,39011,39011,38809,38809
2,02-JAN-18 00:00:00,5956084,41,224,38969,38969,38768,38778
3,02-JAN-18 00:00:00,5956084,40,223,38936,38936,38744,38744
4,02-JAN-18 00:00:00,5956084,39,222,38893,38893,38715,38715


In [7]:
most_common_ob_subroutes = []

for line in all_lines:
    print(line)
    df = pd.read_csv("~/data/stoptimes/" + line + "_stops.csv")
    merged = df.merge(trips_df, how="left", on=["DAYOFSERVICE", "TRIPID"])
    
    subsection = merged.loc[merged["DIRECTION"] == 1] 
    unique_routes = subsection.ROUTEID.unique().tolist()
    
    entries_per_route = {}
    
    for route in unique_routes:
        current = subsection.loc[subsection["ROUTEID"] == route]
        num_entries = current.shape[0]
        entries_per_route[route] = num_entries
    print(entries_per_route)
    
    routes = []
    
    if entries_per_route == {}:
        pass
    else:       
        most_common_route = max(entries_per_route.items(), key=lambda x: x[1])
        
        for key, value in entries_per_route.items():
            if value == most_common_route[1]:
                routes.append(key)
                
    print(routes)
    
    if routes == []:
        pass
    else:
        for route in routes:
            most_common_ob_subroutes.append(route)

1
{'1_37': 596936, '1_39': 8045, '1_38': 45847}
['1_37']
102
{'102_8': 487108}
['102_8']
104
{'104_15': 130510}
['104_15']
11
{'11_40': 778008, '11_41': 7181}
['11_40']
111
{'111_7': 178984}
['111_7']
114
{'114_5': 139653}
['114_5']
116
{'116_1': 8107}
['116_1']
118
{}
[]
120
{'120_7': 316852, '120_8': 25825, '120_12': 75618}
['120_7']
122
{'122_14': 428181, '122_15': 3556, '122_18': 474255, '122_19': 3472}
['122_18']
123
{'123_34': 1005227, '123_35': 9686}
['123_34']
13
{'13_60': 640002, '13_65': 132100, '13_61': 597793, '13_66': 114393, '13_63': 150165, '13_64': 9343, '13_62': 36177, '13_28': 1771}
['13_60']
130
{'130_10': 998538}
['130_10']
14
{'14_15': 1356188}
['14_15']
140
{'140_19': 944591, '140_20': 8751, '140_16': 126}
['140_19']
142
{'142_8': 27035, '142_10': 8254, '142_13': 41102}
['142_13']
145
{'145_102': 1959988, '145_95': 13629, '145_104': 8652, '145_94': 4483, '145_99': 32848, '145_107': 5465}
['145_102']
14C
{'14C_17': 10631}
['14C_17']
15
{'15_16': 1535439, '15_10': 9

In [8]:
print(most_common_ob_subroutes)

['1_37', '102_8', '104_15', '11_40', '111_7', '114_5', '116_1', '120_7', '122_18', '123_34', '13_60', '130_10', '14_15', '140_19', '142_13', '145_102', '14C_17', '15_16', '150_8', '151_15', '15A_83', '15B_60', '15D_62', '16_20', '161_50', '16C_29', '16D_30', '17_10', '17A_15', '18_3', '184_29', '185_53', '220_10', '236_9', '238_11', '239_26', '25_272', '25A_273', '25B_274', '25D_277', '25X_10', '26_28', '27_19', '270_42', '27A_4', '27B_23', '27X_43', '29A_14', '31_15', '31A_25', '31B_44', '31D_50', '32_57', '32X_77', '33_44', '33A_22', '33B_56', '33D_61', '33E_74', '33X_46', '37_14', '38_9', '38A_20', '38B_40', '38D_68', '39_20', '39A_40', '39X_1', '4_10', '40_27', '40B_63', '40D_102', '40E_90', '41_3', '41B_52', '41C_78', '41D_26', '41X_121', '42_42', '42D_50', '43_84', '44_40', '44B_60', '45A_60', '46A_74', '47_139', '49_27', '51D_50', '53_20', '54A_11', '56A_31', '59_10', '61_106', '63_26', '65_74', '65B_65', '66_17', '66A_37', '66B_58', '66X_92', '67_4', '67X_43', '68_81', '68A_86'

In [1]:
file = open("stoptimes/most_common_ob_subroutes.txt", "w")
for element in most_common_ob_subroutes:
    file.write(element + "\n")
file.close()

In [10]:
most_common_ib_subroutes = []

for line in all_lines:
    print(line)
    df = pd.read_csv("~/data/stoptimes/" + line + "_stops.csv")
    merged = df.merge(trips_df, how="left", on=["DAYOFSERVICE", "TRIPID"])
    
    subsection = merged.loc[merged["DIRECTION"] == 2] 
    unique_routes = subsection.ROUTEID.unique().tolist()
    
    entries_per_route = {}
    
    for route in unique_routes:
        current = subsection.loc[subsection["ROUTEID"] == route]
        num_entries = current.shape[0]
        entries_per_route[route] = num_entries
    print(entries_per_route)
    
    routes = []
    
    if entries_per_route == {}:
        pass
    else:       
        most_common_route = max(entries_per_route.items(), key=lambda x: x[1])
    
        for key, value in entries_per_route.items():
            if value == most_common_route[1]:
                routes.append(key)
                
    print(routes)
    
    if routes == []:
        pass
    else:
        for route in routes:
            most_common_ib_subroutes.append(route)
            

1
{'1_40': 639503, '1_41': 46629}
['1_40']
102
{'102_10': 32183, '102_9': 479448}
['102_9']
104
{'104_16': 135084}
['104_16']
11
{'11_42': 852643, '11_43': 14807}
['11_42']
111
{'111_8': 133661, '111_9': 32049, '111_10': 3553}
['111_8']
114
{'114_6': 141607}
['114_6']
116
{'116_3': 14387}
['116_3']
118
{'118_3': 3348, '118_4': 10619}
['118_4']
120
{'120_9': 300878, '120_10': 4110, '120_11': 20514, '120_13': 70035}
['120_9']
122
{'122_16': 443962, '122_17': 5160, '122_20': 480562, '122_21': 4954}
['122_20']
123
{'123_36': 1087313, '123_37': 5424}
['123_36']
13
{'13_67': 768649, '13_71': 17014, '13_68': 565254, '13_73': 55600, '13_76': 18261, '13_70': 125302, '13_69': 35413, '13_72': 78322, '13_75': 303, '13_74': 19507}
['13_67']
130
{'130_11': 1015515}
['130_11']
14
{'14_16': 1294531, '14_12': 24390, '14_13': 3301}
['14_16']
140
{'140_21': 904012, '140_22': 6591}
['140_21']
142
{'142_9': 23988, '142_12': 46874}
['142_12']
145
{'145_105': 1930693, '145_91': 77923, '145_93': 17283, '145_8

In [11]:
print(most_common_ib_subroutes)

['1_40', '102_9', '104_16', '11_42', '111_8', '114_6', '116_3', '118_4', '120_9', '122_20', '123_36', '13_67', '130_11', '14_16', '140_21', '142_12', '145_105', '14C_18', '15_17', '150_9', '151_17', '15A_84', '15B_61', '15D_63', '16_24', '161_51', '16C_28', '17_15', '17A_17', '18_4', '184_28', '185_56', '220_12', '236_10', '238_15', '239_28', '25_269', '25A_270', '25B_271', '25D_275', '25X_11', '26_29', '27_17', '270_44', '27A_5', '27B_34', '27X_42', '29A_15', '31_18', '31A_26', '31B_46', '31D_51', '32_58', '32X_76', '33_70', '33A_25', '33B_58', '33D_62', '33X_49', '37_15', '38_14', '38A_32', '38B_44', '38D_61', '39_21', '39A_43', '39X_3', '4_12', '40_31', '40B_65', '40D_104', '40E_91', '41_7', '41A_22', '41B_53', '41C_79', '41D_27', '41X_125', '42_44', '42D_51', '43_88', '44_36', '44B_61', '45A_64', '46A_67', '46E_94', '47_136', '49_28', '51D_51', '51X_64', '53_21', '54A_12', '56A_30', '59_11', '61_108', '63_30', '65_77', '65B_66', '66_18', '66A_38', '66B_59', '66X_102', '67_6', '67X_

In [2]:
file = open("stoptimes/most_common_ib_subroutes.txt", "w")
for element in most_common_ib_subroutes:
    file.write(element + "\n")
file.close()

In [10]:
with open("stoptimes/most_common_ob_subroutes.txt") as file:
    most_common_ob_subroutes = file.readlines()
most_common_ob_subroutes = [line.strip() for line in most_common_ob_subroutes] 

In [12]:
with open("stoptimes/most_common_ib_subroutes.txt") as file:
    most_common_ib_subroutes = file.readlines()
most_common_ib_subroutes = [line.strip() for line in most_common_ib_subroutes] 

In [2]:
with open("stoptimes/most_common_subroutes.txt") as file:
    most_common_subroutes = file.readlines()
most_common_subroutes = [line.strip() for line in most_common_subroutes] 

In [3]:
most_common_subroutes.sort()
print(most_common_subroutes)

['102_8', '102_9', '104_15', '104_16', '111_7', '111_8', '114_5', '114_6', '116_1', '116_3', '118_4', '11_40', '11_42', '120_7', '120_9', '122_18', '122_20', '123_34', '123_36', '130_10', '130_11', '13_60', '13_67', '140_19', '140_21', '142_12', '142_13', '145_102', '145_105', '14C_17', '14C_18', '14_15', '14_16', '150_8', '150_9', '151_15', '151_17', '15A_83', '15A_84', '15B_60', '15B_61', '15D_62', '15D_63', '15_16', '15_17', '161_50', '161_51', '16C_28', '16C_29', '16D_30', '16_20', '16_24', '17A_15', '17A_17', '17_10', '17_15', '184_28', '184_29', '185_53', '185_56', '18_3', '18_4', '1_37', '1_40', '220_10', '220_12', '236_10', '236_9', '238_11', '238_15', '239_26', '239_28', '25A_270', '25A_273', '25B_271', '25B_274', '25D_275', '25D_277', '25X_10', '25X_11', '25_269', '25_272', '26_28', '26_29', '270_42', '270_44', '27A_4', '27A_5', '27B_23', '27B_34', '27X_42', '27X_43', '27_17', '27_19', '29A_14', '29A_15', '31A_25', '31A_26', '31B_44', '31B_46', '31D_50', '31D_51', '31_15', '3

In [3]:
file = open("stoptimes/most_common_subroutes.txt", "w")
for element in most_common_subroutes:
    file.write(element + "\n")
file.close()

In [7]:
longest_subroutes_df = pd.read_csv("~/data/stoptimes/stops_per_longest_subroutes.csv")

In [8]:
longest_subroutes = longest_subroutes_df.ROUTEID.unique().tolist()
longest_subroutes.sort()
print(longest_subroutes)

['102_8', '102_9', '104_15', '104_16', '111_7', '111_8', '111_9', '114_5', '114_6', '116_1', '116_3', '118_4', '11_40', '11_42', '120_10', '120_11', '120_8', '122_14', '122_16', '122_18', '122_20', '123_34', '123_36', '130_10', '130_11', '13_61', '13_63', '13_67', '13_68', '13_70', '140_19', '140_21', '142_12', '142_13', '145_102', '145_105', '145_93', '14C_17', '14C_18', '14_15', '14_16', '150_8', '150_9', '151_15', '151_16', '151_17', '15A_83', '15A_84', '15B_56', '15B_60', '15B_61', '15D_62', '15D_63', '15_16', '15_17', '161_50', '161_51', '16C_28', '16C_29', '16D_30', '16_20', '16_24', '17A_11', '17A_13', '17A_15', '17A_17', '17_10', '17_15', '184_28', '184_29', '185_54', '185_55', '185_59', '185_60', '185_62', '18_3', '18_4', '1_37', '1_40', '220_14', '220_15', '236_10', '236_9', '238_11', '238_15', '239_26', '239_28', '25A_270', '25A_273', '25B_271', '25B_274', '25D_275', '25D_277', '25X_10', '25X_9', '25_269', '25_272', '26_24', '26_26', '26_28', '26_29', '270_42', '270_44', '27

In [9]:
len(most_common_subroutes)

252

In [10]:
len(longest_subroutes)

293

In [11]:
most_common_NOT_longest = np.setdiff1d(most_common_subroutes, longest_subroutes).tolist()
print("Most common subroutes that are NOT also longest subroutes:\n", most_common_NOT_longest)

Most common subroutes that are NOT also longest subroutes:
 ['120_7', '120_9', '13_60', '185_53', '185_56', '220_10', '220_12', '25X_11', '27A_5', '31_15', '33X_49', '33_44', '38A_32', '38_14', '38_9', '39_20', '40D_102', '40D_104', '41B_52', '41X_125', '41_3', '41_7', '45A_60', '45A_64', '47_136', '47_139', '49_28', '51X_64', '53_20', '61_106', '61_108', '63_26', '63_30', '65_74', '65_77', '66X_102', '67X_46', '68_83', '75_17', '75_19', '7A_85', '7_72', '84A_1', '84A_3', '84X_62', '84_27', '84_30']


In [12]:
len(most_common_NOT_longest)

47

In [13]:
longest_NOT_most_common = np.setdiff1d(longest_subroutes, most_common_subroutes).tolist()
print("Longest subroutes that are NOT also most common:\n", longest_NOT_most_common)

Longest subroutes that are NOT also most common:
 ['111_9', '120_10', '120_11', '120_8', '122_14', '122_16', '13_61', '13_63', '13_68', '13_70', '145_93', '151_16', '15B_56', '17A_11', '17A_13', '185_54', '185_55', '185_59', '185_60', '185_62', '220_14', '220_15', '25X_9', '26_24', '26_26', '27A_7', '31_17', '32X_74', '33X_47', '33_69', '38A_33', '38_10', '38_15', '39X_2', '39_22', '40D_101', '40D_103', '41B_58', '41C_83', '41X_126', '41X_128', '41_22', '41_25', '43_85', '43_89', '45A_67', '45A_69', '46E_93', '47_135', '47_138', '49_29', '4_14', '51D_47', '51X_63', '53_22', '61_107', '61_109', '63_27', '63_31', '65_73', '65_76', '66A_34', '66A_36', '66B_55', '66B_57', '66X_97', '66_11', '66_13', '67X_41', '68_78', '68_82', '69_43', '69_44', '75_18', '75_21', '77A_30', '77A_31', '77X_62', '7A_86', '7A_89', '7_66', '7_67', '7_74', '84A_2', '84A_4', '84X_53', '84_28', '84_31']


In [14]:
len(longest_NOT_most_common)

88

In [15]:
mylist = list(dict.fromkeys(longest_NOT_most_common))

In [16]:
len(mylist)

88